In [4]:
# tf-idf preprocess
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from scipy import sparse
train_merged=pd.read_csv('train_merged.csv')
df=train_merged
v = TfidfVectorizer(max_df=50,lowercase=False,tokenizer=lambda x:x)
cur=df.groupby("user_id")["creative_id"].apply(list)
# cur.astype(int)
display(cur)
v_fit = v.fit_transform(cur)
display(v_fit)
sparse.save_npz('v_fit.npz', v_fit)

user_id
1         [71691, 90171, 122032, 209778, 821396, 877468,...
2         [13069, 15558, 22013, 39714, 63441, 96192, 155...
3         [66009, 72533, 392052, 593522, 599128, 661347,...
4         [31070, 39588, 72989, 215041, 574787, 589886, ...
5         [24333, 43235, 75011, 296145, 350759, 795508, ...
                                ...                        
899996    [92193, 114074, 654526, 931537, 939409, 145880...
899997    [7400, 24333, 30210, 60330, 69204, 103918, 103...
899998    [71752, 648402, 1251649, 1370659, 1779124, 230...
899999    [12838, 12838, 12838, 12838, 122834, 279342, 3...
900000    [234063, 285502, 423617, 1224107, 1405713, 180...
Name: creative_id, Length: 900000, dtype: object

<900000x2414321 sparse matrix of type '<class 'numpy.float64'>'
	with 9965541 stored elements in Compressed Sparse Row format>

In [ ]:
# cross validation for age
%reset -f
# !pip install lightgbm
import lightgbm as lgb
import pandas as pd
from scipy import sparse
user_info=pd.read_csv('user_info.csv')
agg=pd.read_csv('agg.csv')
v_fit=sparse.load_npz('v_fit.npz')
num_train=int(user_info.shape[0]*0.8)
y_train=user_info["age"].iloc[:num_train]
y_test=user_info["age"].iloc[num_train:]
X_train=sparse.csr_matrix(agg.iloc[:num_train])
X_test=sparse.csr_matrix(agg.iloc[num_train:])
X_train=sparse.hstack((v_fit[:num_train,:],X_train),dtype=float).tocsr()
X_test=sparse.hstack((v_fit[num_train:,:],X_test),dtype=float).tocsr()
train_data = lgb.Dataset(X_train,y_train)
train_data.save_binary('CountVectorizer_train_age.bin')

del 
lgb_params = {'num_leaves': 2**6-1,
              'min_data_in_leaf': 25, 
              'objective':'multiclass',
              'num_class':10,
              'max_depth': -1,
              'learning_rate': 0.1,
              'boosting': 'gbdt',
              'feature_fraction': 0.7,
              'bagging_fraction': 0.9,
              'bagging_seed': 11,
              'metric': 'multi_error',
              'seed':1024,
              'nthread':12,
             }
print("start training")
cv_results = lgb.cv(
    lgb_params, train_data, num_boost_round=1000, nfold=5, stratified=False, shuffle=True,
    verbose_eval=10, show_stdv=True, seed=0)

start training


In [13]:
# predict age
%reset -f
import lightgbm as lgb
train_data = lgb.Dataset('CountVectorizer_train_age.bin')
lgb_params = {'num_leaves': 2**6-1,
              'min_data_in_leaf': 25, 
              'objective':'multiclass',
              'num_class':10,
              'max_depth': -1,
              'learning_rate': 0.1,
              'boosting': 'gbdt',
              'feature_fraction': 0.6,
              'bagging_fraction': 0.9,
              'bagging_seed': 11,
              'metric': 'multi_error',
              'seed':1024,
              'nthread':12,
              'lambda_l1': 0.2,
             }
print("start training")
model = lgb.train(lgb_params, train_data,num_boost_round=200)
model.save_model("age_model.txt")
print("training completed!")

start training


KeyboardInterrupt: 

In [11]:
# predict age
%reset -f
import pandas as pd
from scipy import sparse
import lightgbm as lgb
from sklearn import metrics
user_info=pd.read_csv('user_info.csv')
user_info.astype(int)
agg=pd.read_csv('agg.csv')
v_fit=sparse.load_npz('v_fit.npz')
num_train=int(user_info.shape[0]*0.8)
y_test=user_info["age"].iloc[num_train:]
display(y_test)
X_test=sparse.csr_matrix(agg.iloc[num_train:])
X_test=sparse.hstack((v_fit[num_train:,:],X_test),dtype=float).tocsr()
print("start predicting")
model=lgb.Booster(model_file='age_model.txt')
y_pred=model.predict(X_test)
display(y_pred)
print("acc:",metrics.accuracy_score(y_test,y_pred))
print("acc:",metrics.roc_auc_score(y_test,y_pred))

720000    3
720001    2
720002    2
720003    3
720004    5
         ..
899995    5
899996    3
899997    4
899998    3
899999    3
Name: age, Length: 180000, dtype: int64

start predicting


array([3.74877228, 4.60749918, 4.16929228, ..., 4.28402148, 4.44051075,
       4.15519538])

ValueError: Classification metrics can't handle a mix of multiclass and continuous targets

In [ ]:
y_pred=

In [6]:
df=pd.DataFrame(y_pred)

In [8]:
df.describe()

,0
count,180000.0
mean,1.0
std,0.0
min,1.0
25%,1.0
50%,1.0
75%,1.0
max,1.0


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
df=tmpdf
features=["creative_id"]
v = TfidfVectorizer(max_df=1.0,min_df=1,lowercase=False,tokenizer=lambda x:x)
for feature in features:
    cur=df.groupby("user_id")[feature].apply(list)
    v_fit = v.fit_transform(cur)
    word=v.get_feature_names()
    display(v_fit)
    display(type(v_fit))
#     for i in range(len(weight)):
#         print("-----output feature={}, user_id={} tf-idf-----".format(feature,i+1))
#         for j in range(len(word)):
#             print(word[j],weight[i][j])